# Dask vs Pandas speed tests



Dask based on Tornado and first release was in 2015 (before asyncio)

In [ ]:
import pandas as pd
from pathlib import Path
import dask.dataframe as dd
# from dask.diagnostics import ProgressBar  # single machine progressbar
# from dask.distributed import progress  # does it work?
from dask.distributed import Client  # has dashboard

Articles:
+ CSV to Parquet with Dask: <https://mungingdata.com/dask/read-csv-to-parquet/>
+ https://towardsdatascience.com/beyond-pandas-spark-dask-vaex-and-other-big-data-technologies-battling-head-to-head-a453a1f8cc13
+ https://towardsdatascience.com/make-your-data-processing-code-fly-in-5-minutes-c4998e6da094
+ https://stackoverflow.com/questions/47191675/pandas-write-dataframe-to-parquet-format-with-append

Issues:
+ https://stackoverflow.com/questions/60173358/distributed-worker-memory-use-is-high-but-worker-has-no-data-to-store-to-disk
+ https://github.com/dask/distributed/issues/4594

Results:
+ no reindex for dask
+ No "Try using .loc[row_indexer,col_indexer] = value instead" warning in dask
+ No append for parquet for both (possible with pandas and pyarrow)
+ CSV:
    + no gzip input for CSV in dask
    + 35 min (dask csv -> csv) vs 55 min (pandas gz -> gz) vs 55 min (pd csv -> csv)
    + 550 output csv files (56GB) vs 1 (5GB)
+ parquet:
    + dask + pyarrow is not working, but takes 11 min
    + dask + fastparquet 14 min, 554 files (58GB) (compression unknown) vs 33 files
    + pandas ~27 min (csv -> fastparquet) (snappy, ~7GB)
    + pandas ~37 min (csv -> fastparquet) (gzip, ~3.7GB)
    + pandas ~25 min (csv -> fastparquet) (uncompressed, 45GB)
    + **pandas ~20 min (csv -> pyarrow) (snappy, ~5GB)**
    + pandas ~28 min (csv -> pyarrow) (gzip, ~3.7GB)
    + pandas ~33 min (csv -> pyarrow) (brotli, ~3GB)
    + **pandas 20 min (*gz* -> pyarrow) (snappy, 5GB)** vs 33 files 

In [15]:
data_path = './data/mimic-iii-clinical-database-1.4/'
output_path = './data/fhir_out/'

## Pandas

In [ ]:
from pathlib import Path
import time
import gc


def transform_chartevents(data_path, output_path, chunksize=10**7):
    """ ~6GB RAM in peak consumption with default chunksize
    """
    # delete outputfile if exists
    output_filename = output_path+'observation_ce'
    Path(output_filename).unlink(missing_ok=True)
    
    d_items = pd.read_csv(data_path+'D_ITEMS.csv.gz', index_col=0,
                      # dropped 'ABBREVIATION', 'LINKSTO', 'CONCEPTID', 'UNITNAME'
                      usecols=['ROW_ID', 'ITEMID', 'LABEL', 'DBSOURCE', 'CATEGORY', 'PARAM_TYPE'],
                      dtype={'ROW_ID': int, 'ITEMID': int, 'LABEL': str, 'DBSOURCE': 'category',
                             'CATEGORY': 'category', 'PARAM_TYPE': str})
    
    # it is the biggest file ~4GB gzipped, 33GB unpacked, 330M strings
    # looks like CareVue and Metavision data should be processed separately
    chunk_container =  pd.read_csv(data_path+'CHARTEVENTS.csv.gz',
                                   # STORETIME
                                   usecols=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME',
                                            'STORETIME', 'CGID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING', 'ERROR',
                                            'RESULTSTATUS', 'STOPPED'],
                                   dtype={'ROW_ID': int, 'SUBJECT_ID': int, 'HADM_ID': int, 'ICUSTAY_ID': float,
                                          'ITEMID': int, 'CGID': float, 'VALUE': str, 'VALUENUM': float, 
                                          'VALUEUOM': str, 'WARNING': float, 'ERROR': float,
                                          'RESULTSTATUS': str, 'STOPPED': str},
                                   parse_dates=['CHARTTIME'],
                                   chunksize=chunksize)  # 2.67GB for 10**7
    
    for i, chartevents in enumerate(chunk_container):
        # Show progress (~330M strings)
        print(f'{i + 1}/{330*10**6 / chunksize}', flush=True, end =" ")
        start_time = time.time()

        observation_ce = pd.merge(chartevents, d_items, on='ITEMID')

        observation_ce['note'] = observation_ce['LABEL'].str.cat(observation_ce['DBSOURCE'], sep=' ', na_rep='NA')
        observation_ce['note'] = observation_ce['note'].str.cat(observation_ce['PARAM_TYPE'], sep=' ', na_rep='')

        observation_ce.loc[observation_ce['STOPPED'] == "D/C'd", 'RESULTSTATUS'] = 'discharged'
        observation_ce.loc[observation_ce['ERROR'] == 1, 'RESULTSTATUS'] = 'Error'
        
        # New columns to adapt to Chartevents observations
        observation_ce['category'] = 'chartevents'  # ????

        observation_ce.drop(['LABEL', 'PARAM_TYPE', 'ERROR', 'DBSOURCE', 'STOPPED'], axis=1, inplace=True)

        observation_ce.rename(columns={'ROW_ID':'identifier',
                                       'SUBJECT_ID':'subject',
                                       'HADM_ID':'encounter',                               
                                       'ICUSTAY_ID':'partOf',
                                       'ITEMID':'code',
                                       'CGID':'performer',
                                       'CHARTTIME':'effectiveDateTime',
                                       'VALUE':'value',
                                       'VALUENUM':'value_quantity',
                                       'VALUEUOM':'unit',
                                       'WARNING':'interpretation',
                                       'RESULTSTATUS':'status',
                                       'CATEGORY':'category_sub'}, inplace=True)

        observation_ce = observation_ce.reindex(columns=['identifier',
                                                         'subject', 
                                                         'encounter', 
                                                         'partOf', 
                                                         'code',
                                                         'effectiveDateTime',
                                                         'performer',
                                                         'value',
                                                         'value_quantity',
                                                         'unit', 
                                                         'interpretation',
                                                         'status',
                                                         'note',
                                                         'category_sub',
                                                         'category'], copy=False)

#         observation_ce.to_csv(output_filename + '.csv.gz',
#                               compression={'method': 'gzip', 'compresslevel': 1},
#                               index=False, mode='a')
        
#         observation_ce.to_csv(output_filename + '.csv', index=False, mode='a')
        
        # will create a lot of files, no append mode
        observation_ce.to_parquet(f"{output_filename}_{i}.parquet", 
                                  compression='snappy', index=False)

        # force free mem, for some reasons without it, RAM ends pretty quick
        gc.collect()
        # show execution time per chunk
        print(f"--- {time.time() - start_time} seconds ---", flush=True)

        
transform_chartevents(data_path, output_path)
# observation_ce = transform_chartevents(data_path, output_path)
# observation_ce.head()

## Dask

In [ ]:
d_items = pd.read_csv(data_path+'D_ITEMS.csv.gz', index_col=0,
                  # dropped 'ABBREVIATION', 'LINKSTO', 'CONCEPTID', 'UNITNAME'
                  usecols=['ROW_ID', 'ITEMID', 'LABEL', 'DBSOURCE', 'CATEGORY', 'PARAM_TYPE'],
                  dtype={'ROW_ID': int, 'ITEMID': int, 'LABEL': str, 'DBSOURCE': 'category',
                         'CATEGORY': 'category', 'PARAM_TYPE': str})

d_items.set_index('ITEMID', inplace=True)
d_items.head()

In [ ]:
# client = Client(threads_per_worker=2, n_workers=4, memory_limit='2.5GB')
client = Client()  # it will consume all memory by default
client

In [ ]:
# Warning gzip compression does not support breaking apart files
# Q: what default blocksize? A: https://github.com/dask/dask/pull/1328 (32M)
# max 64e6 bytes ~61MB
df =  dd.read_csv(data_path + 'CHARTEVENTS' + '.csv',
                  usecols=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME',
                           'STORETIME', 'CGID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING', 'ERROR',
                           'RESULTSTATUS', 'STOPPED'],
                  dtype={'ROW_ID': int, 'SUBJECT_ID': int, 'HADM_ID': int, 'ICUSTAY_ID': float,
                         'ITEMID': int, 'CGID': float, 'VALUE': str, 'VALUENUM': float, 
                         'VALUEUOM': str, 'WARNING': float, 'ERROR': float,
                         'RESULTSTATUS': str, 'STOPPED': str},
                  parse_dates=['CHARTTIME'])
#                   blocksize=96*1024*1024)  # 64MB max?

# it will take a while, like 4min
# raises "TypeError: memoryview: cannot cast view with zeros in shape or strides"
# at the final stages
# df = df.set_index('ITEMID')

In [ ]:
observation_ce = dd.merge(df, d_items, left_on='ITEMID', right_index=True)

In [ ]:
output_filename = output_path+'observation_ce'

observation_ce['note'] = observation_ce['LABEL'].str.cat(observation_ce['DBSOURCE'], sep=' ', na_rep='NA')
observation_ce['note'] = observation_ce['note'].str.cat(observation_ce['PARAM_TYPE'], sep=' ', na_rep='')

# No "Try using .loc[row_indexer,col_indexer] = value instead" warning
# .loc way is not implemented/usable in dask
mask = observation_ce['STOPPED'] == "D/C'd"
observation_ce[mask]['RESULTSTATUS'] = 'discharged'
mask2 = observation_ce['ERROR'] == 1
observation_ce[mask]['RESULTSTATUS'] = 'Error'

# New columns to adapt to Chartevents observations
observation_ce['category'] = 'chartevents'  # ????

observation_ce.drop(columns=['LABEL', 'PARAM_TYPE', 'ERROR', 'DBSOURCE', 'STOPPED'])

observation_ce.rename(columns={'ROW_ID':'identifier',
                               'SUBJECT_ID':'subject',
                               'HADM_ID':'encounter',                               
                               'ICUSTAY_ID':'partOf',
                               'ITEMID':'code',
                               'CGID':'performer',
                               'CHARTTIME':'effectiveDateTime',
                               'VALUE':'value',
                               'VALUENUM':'value_quantity',
                               'VALUEUOM':'unit',
                               'WARNING':'interpretation',
                               'RESULTSTATUS':'status',
                               'CATEGORY':'category_sub'})

# dask loves parquet
# using pyarrow: https://github.com/dask/dask/issues/6587
# observation_ce.to_csv(output_filename + '.csv')
observation_ce.to_parquet(output_filename + '.parquet', schema="infer")

Attemt to use `map_partition(foo)` failed with issues from header

In [ ]:
# https://stackoverflow.com/questions/41806850/dask-difference-between-client-persist-and-client-compute
# observation_ce.persist()  # in bg
# observation_ce.compute()

## parquet to single file

+ https://arrow.apache.org/docs/python/dataset.html
+ https://stackoverflow.com/questions/47191675/pandas-write-dataframe-to-parquet-format-with-append
+ **https://stackoverflow.com/questions/47113813/using-pyarrow-how-do-you-append-to-parquet-file**

### use dataset

it need to be dowloaded into the memory in order to save as a single file

https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Dataset.html#pyarrow.dataset.Dataset.to_table
>Note that this method reads all the selected data from the dataset into memory.

In [28]:
import pandas as pd 
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.dataset as ds

data_path = './data/mimic-iii-clinical-database-1.4/'
output = "mydataset.parquet"

d_items = pd.read_csv(data_path+'D_ITEMS.csv.gz', index_col=0,
                  # dropped 'ABBREVIATION', 'LINKSTO', 'CONCEPTID', 'UNITNAME'
                  usecols=['ROW_ID', 'ITEMID', 'LABEL', 'DBSOURCE', 'CATEGORY', 'PARAM_TYPE'],
                  dtype={'ROW_ID': int, 'ITEMID': int, 'LABEL': str, 'DBSOURCE': str,
                         'CATEGORY': str, 'PARAM_TYPE': str}, chunksize=1000)

for i, chunk in enumerate(d_items):
    # create a parquet table from your dataframe
    table = pa.Table.from_pandas(chunk)
    if i == 0:
        schema = table.schema
    # write direct to your parquet file
    pq.write_to_dataset(table, root_path=output)

In [29]:
# tru to convert dataset to a single file
# dataset = ds.dataset(output, schema=schema)
dataset = ds.dataset(output)

In [30]:
# it will load all into the memory
dataset.to_table()
# pq.read_table(output)

pyarrow.Table
ITEMID: int64
LABEL: string
DBSOURCE: string
CATEGORY: string
PARAM_TYPE: string
ROW_ID: int64

In [31]:
 print(dataset.schema.to_string())

ITEMID: int64
  -- field metadata --
  PARQUET:field_id: '1'
LABEL: string
  -- field metadata --
  PARQUET:field_id: '2'
DBSOURCE: string
  -- field metadata --
  PARQUET:field_id: '3'
CATEGORY: string
  -- field metadata --
  PARQUET:field_id: '4'
PARAM_TYPE: string
  -- field metadata --
  PARQUET:field_id: '5'
ROW_ID: int64
  -- field metadata --
  PARQUET:field_id: '6'
-- schema metadata --
pandas: '{"index_columns": ["ROW_ID"], "column_indexes": [{"name": null, ' + 881


In [32]:
 print(schema.to_string())

ITEMID: int64
LABEL: string
DBSOURCE: string
CATEGORY: string
PARAM_TYPE: null
ROW_ID: int64
-- schema metadata --
pandas: '{"index_columns": ["ROW_ID"], "column_indexes": [{"name": null, ' + 879


### use single file from start

In [16]:
chunksize=10000 # this is the number of lines

pqwriter = None
for i, df in enumerate(pd.read_csv(data_path+'D_ITEMS.csv.gz', index_col=0,
                  # dropped 'ABBREVIATION', 'LINKSTO', 'CONCEPTID', 'UNITNAME'
                  usecols=['ROW_ID', 'ITEMID', 'LABEL', 'DBSOURCE', 'CATEGORY', 'PARAM_TYPE'],
                  dtype={'ROW_ID': int, 'ITEMID': int, 'LABEL': str, 'DBSOURCE': str,
                         'CATEGORY': str, 'PARAM_TYPE': str}, chunksize=chunksize)):
    table = pa.Table.from_pandas(df)
    # for the first chunk of records
    if i == 0:
        # create a parquet write object giving it an output file
        pqwriter = pq.ParquetWriter('sample.parquet', table.schema)            
    pqwriter.write_table(table)

# close the parquet writer
if pqwriter:
    pqwriter.close()

In [17]:
pd.read_parquet('sample.parquet')

,ITEMID,LABEL,DBSOURCE,CATEGORY,PARAM_TYPE
ROW_ID,,,,,
457,497,Patient controlled analgesia (PCA) [Inject],carevue,None,None
458,498,PCA Lockout (Min),carevue,None,None
459,499,PCA Medication,carevue,None,None
460,500,PCA Total Dose,carevue,None,None
461,501,PCV Exh Vt (Obser),carevue,None,None
...,...,...,...,...,...
14518,226757,GCSMotorApacheIIValue,metavision,Scores - APACHE II,Text
14519,226758,GCSVerbalApacheIIValue,metavision,Scores - APACHE II,Text
14520,226759,HCO3ApacheIIValue,metavision,Scores - APACHE II,Numeric


In [18]:
pd.read_csv(data_path+'D_ITEMS.csv.gz', index_col=0,
                  # dropped 'ABBREVIATION', 'LINKSTO', 'CONCEPTID', 'UNITNAME'
                  usecols=['ROW_ID', 'ITEMID', 'LABEL', 'DBSOURCE', 'CATEGORY', 'PARAM_TYPE'],
                  dtype={'ROW_ID': int, 'ITEMID': int, 'LABEL': str, 'DBSOURCE': 'category',
                         'CATEGORY': 'category', 'PARAM_TYPE': str})

,ITEMID,LABEL,DBSOURCE,CATEGORY,PARAM_TYPE
ROW_ID,,,,,
457,497,Patient controlled analgesia (PCA) [Inject],carevue,NaN,NaN
458,498,PCA Lockout (Min),carevue,NaN,NaN
459,499,PCA Medication,carevue,NaN,NaN
460,500,PCA Total Dose,carevue,NaN,NaN
461,501,PCV Exh Vt (Obser),carevue,NaN,NaN
...,...,...,...,...,...
14518,226757,GCSMotorApacheIIValue,metavision,Scores - APACHE II,Text
14519,226758,GCSVerbalApacheIIValue,metavision,Scores - APACHE II,Text
14520,226759,HCO3ApacheIIValue,metavision,Scores - APACHE II,Numeric


In [12]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pandas as pd
import numpy as np


df = pd.DataFrame({'A': ['1', '2', '3', np.nan], 'B': ['1', '2', '3', np.nan]}, dtype=str).to_csv('tmp.csv')

for i, chunk in enumerate(pd.read_csv('tmp.csv', dtype={'A': str, 'B': str}, chunksize=1)):
    # create a parquet table from your dataframe
    table = pa.Table.from_pandas(chunk)
    if i == 0:
        schema = table.schema
    # write direct to your parquet file
    pq.write_to_dataset(table, root_path='./tmp.parquet')

In [13]:
# dataset = ds.dataset('./tmp.parquet', schema=schema)
dataset = ds.dataset('./tmp.parquet')
dataset.to_table().to_pandas()

,Unnamed: 0,A,B
0,0.0,1,1
1,1.0,2,2
2,NaN,3,3
3,NaN,1,1
4,NaN,2,2
5,NaN,None,None
6,3.0,None,None
7,2.0,3,3


In [11]:
# print(dataset.schema.to_string())
pd.read_csv('tmp.csv')

,A,B
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,NaN,NaN
